In [22]:
from pixell import powspec

In [1]:
cmb_sets = [0]
print(type(cmb_sets))

<class 'list'>


In [24]:
for cmb_set in cmb_sets:
    print(cmb_set)

0


In [25]:
lens_ps = '../inputPs/cosmo2017_10K_acc3_lenspotentialCls.dat'

In [26]:
ps = powspec.read_camb_full_lens(lens_ps)

In [29]:
ps.shape

(4, 4, 10001)

In [31]:
ps[0].shape

(4, 10001)

In [18]:
ps[0, 1:, :] = 0.

In [19]:
ps.shape

(4, 4, 10001)

In [20]:
ps

array([[[ 0.00000000e+00,  0.00000000e+00,  9.09159461e-09, ...,
          1.79102821e-26,  1.78911783e-26,  1.78720876e-26],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 0.00000000e+00,  0.00000000e+00,  1.31487009e-03, ...,
          1.49801990e-20,  1.49681643e-20,  1.49561358e-20],
        [ 0.00000000e+00,  0.00000000e+00,  1.10783029e+03, ...,
          9.53193823e-11,  9.52060612e-11,  9.50990661e-11],
        [ 0.00000000e+00,  0.00000000e+00,  3.13049236e+00, ...,
          1.16852035e-12,  1.16237986e-12,  1.15630463e-12],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000